# 【Task2 GBDT算法梳理】
【参考框架】欢迎有自己的框架
1,GBDT
2,前向分布算法
3,负梯度拟合
4,损失函数
5,回归
6,二分类，多分类
7,正则化
8,优缺点
9,sklearn参数
10,应用场景

In [ ]:
'''

1,GBDT：

Gradient Boosting Decision Tree（译：梯度、变化率、梯度变化曲线、倾斜, 提升、发展, 决策树）
梯度上升树
集成学习算法的一种，通过多个弱学习器按一定规则结合，达到强学习器效果。
GBDT 采用 boosting方法 逐步提升串行学习
（先训练单个学习器，再根据学习结果调整后续训练样本分布，对做错的加关注，
再调整的基础上训练下一个学习器。提升是加法和前向分布的结合）。
RF是并行bagging学习。
GBDT都是回归树，不是分类树（但也可通过调整用于分类任务）,
RF既可以分类也可以回归。
（分类问题比如不同性别的分类；回归问题比如年龄等连续性数值型数据的变化回归 ）
分类树是通过在每个叶子节点设置阈值来划分分类，选叶子节点的方式是通过信息熵和信息增益
或基尼系数优选最适合的；
回归树的核心是--每棵树是之前所有树结论和的残差（真实值-预测值），
即，再加一个预测值即可得到真实值的累加量。
比如：真实年龄18岁，第一棵树预测了12岁，差6岁，6就是残差（真实18-预测12）。
把第二棵树设为6岁，则树一+树二=真实年龄。
如果树二是5岁，则还有1岁的残差，第三棵树设为1岁，继续学习。
Gradient（译：梯度，变化，倾斜）的梯度变化就体现在GBDT它是算差的，
它的损失函数是以误差作衡量标准，所以求最优就是求损失最小的梯度斜率变化回归问题。
所以它叫梯度提升树，核心在于梯度变化更新回归。

2,前向分布算法：
boosting提升算法是通过加法模型(每个基学习器累加学习)实现。
GBDT的核心思想，如前所说，是算之前树结论和的残差（目的是让之前树结论的和补上一个残差最接近真实值）
所以它算的不是简单的把基学习器结果一个个累加起来，而是根据前面的结果，
每一步只学习一个基函数及其系数，然后逐步逼近优化目标，
算法得到了简化，将同时求解从m=1到M的所有参数优化问题，简化为逐次求解各个参数分别单独优化问题。

优化所有参数与前向分布优化分步参数的公式对比：
见文末

3,负梯度拟合
梯度提升树GBDT用梯度下降的值前面加负号求损失函数最小的过程叫负梯度拟合。
这个梯度就是残差的近似值。使损失函数最小的这个梯度（残差）就是补上它以后最接近真实值（前树和+残差=真实值）。

4,损失函数
不同问题的提升树算法，其主要区别在于损失函数不同。
平方损失函数常用于回归问题，对数、指数损失函数用于分类问题，绝对损失函数用于决策问题。

用于分类：

指数损失函数公式：
L(y,h(x))=exp(−yh(x))
对数损失函数分类（二分类，多元分类）
二分类公式：
L(y,h(x))=log(1+exp(−yh(x)))

用于回归：

均方差：
L(y,h(x))=(y−h(x))^2
绝对损失：
L(y,h(x))=|y−h(x)|
对应的负梯度误差等于sign(y_i−h(x_i))
Huber损失（均方差和绝对损失的折衷）：
对于远离中心的异常点，采用绝对损失，中心附近的点采用均方差。这个界限一般用分位数点度量。
分位数损失：
它对应的是分位数回归的损失函数。

5,回归
步骤：
输入样本；
初始化基学习器；
迭代每个基学习器；
计算当次迭代的负梯度；
更新参数得到强学习器；

6,二分类，多分类
由于分类问题输出的不是连续数值，是离散的类别，这个我们不能根据GBDT算法的核心思想计算树与树之间的差值。
所以需要把损失函数转成指数或对数，把结果变成一个[-1,1]之间的分类概率值，变连续性数值型，
才可以共用计算残差的公式来求负梯度。

二分类GBDT，类似于逻辑回归的对数似然损失函数，公式为：
L(y,h(x))=log(1+exp(−y_h(x)))

多分类：
对于多分类任务，GDBT的做法是采用一对多的策略，
如：每个类别训练M个分类器，K个类别，
通过两层for循环遍历依次为每个分类器（每棵树）拟合K个类别。

7,正则化
GBDT正则化防止过拟和（模型训练效果过优，用于测试集时泛化能力弱，适用覆盖不广泛）
方法一：
给加基学习器加法模型加正则化项中和平衡；
方法二：
用随机梯度提升树（Stochastic Gradient Boosting Tree, SGBT），
即通过子采样比例实现正则化，取一部分样本做GBDT决策树拟合，不用全部样本，
这样可以减少方差，
方差衡量每个值偏离均值的程度，选取的越少方差变化越小，但可能偏离真实值（偏差）越大，
所以样本比例也不宜过小。
//使用了子采样，程序可以通过不同的采样分发到多个任务去做boosting的迭代过程，
//形成新树，实现并行。
方法三：
对每个基学习器CART回归树进行正则化剪枝，去掉不必要的节点，使细化复杂的树简化，泛化能力更强。

8,优缺点
优点：
可以灵活处理各种类型数据（连续值，离散值）
和SVM支持向量机相比，不用多次调参（它的调参是只考虑当前步骤的前向分布算法），也可以有较高的预测准确率；
使用一些健壮的损失函数（如Huber的大区域分位数区分方式），对异常值监测能力很强; 
缺点：
弱学习器之间存在前后依赖关系，难以并行训练数据。
可以通过自采样的SGBT（Stochastic Gradient Boosting Tree随机梯度提升树）
来达到部分并行。

9,sklearn参数
class sklearn.ensemble.GradientBoostingClassifier(
            loss=’deviance’, #损失函数
            learning_rate=0.1,#步长,学习率
            n_estimators=100, #弱学习器个数，也即最大迭代次数
            subsample=1.0,#子采样比例
            criterion=’friedman_mse’,
            min_samples_split=2,#树分裂的最小个数
            min_samples_leaf=1, #【某叶节点上样本的最小数量（少于该值时该节点不再切分）】 
            min_weight_fraction_leaf=0.0, 
            max_depth=3, #树最大深度
            min_impurity_decrease=0.0, 
            min_impurity_split=None, 
            init=None, 
            random_state=None, #随机种子
            max_features=None, #最大特征数
            verbose=0, 
            max_leaf_nodes=None, #最多叶节点数
            warm_start=False, 
            presort=’auto’,
            alpha='0.9'#分位数值,用于Huber)

10,应用场景：

用于所有回归问题，
逻辑回归LR用于线性回归，
GBDT线性非线性都可，
也可用于二分类问题（设定阈值，大于阈值为正例，反之为负例）。


'''


